In [0]:
print("Healthy")

In [0]:
from re import search
from sqlalchemy import create_engine
import duckdb
from datetime import datetime, timedelta

import pandas as pd 
import numpy as np 
import mysql.connector

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')


import json
with open('/Workspace/Credentials/db_data.json', 'r') as fp:
    data = json.load(fp)

Dumpdb = mysql.connector.connect(
  host=data['Dumpdb']['host'],
  user=data['Dumpdb']['user'],
  passwd=data['Dumpdb']['passwd'],
  database = data['Dumpdb']['database']
)


MifosDB = mysql.connector.connect(
  host=data['mifos']['host'],
  user=data['mifos']['user'],
  passwd=data['mifos']['passwd'],
  database = data['mifos']['database']
)


vbizdb = mysql.connector.connect(
host=data['vbizdb']['host'],
  user=data['vbizdb']['user'],
  passwd=data['vbizdb']['passwd'],
  database = data['vbizdb']['database']
)

host = data['redshift']['host']
user = data['redshift']['user']
passwd = data['redshift']['passwd']
database = data['redshift']['database']

conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
pd.set_option('display.float_format', lambda x: '%.2f' % x)



In [0]:
#   DROP TRUNCATE IF EXISTS {schema_name}.{table_name};
def load_table(schema_name, table_name, dataframe):
    engine = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
    
    sql = f'''
    
    -- Check if the table exists in the system catalog
    SELECT COUNT(*)
    FROM pg_tables
    WHERE schemaname = '{schema_name}' -- adjust the schema if needed
    AND tablename = '{table_name}'; -- adjust the table name

    -- Truncate the table if it exists
    TRUNCATE TABLE {schema_name}.{table_name}; -- adjust the table name
    '''

    with engine.begin() as con:     
        con.execute(sql)

    chunk_size = 10000
    num_rows = len(dataframe)

    conn = create_engine(f"postgresql+psycopg2://{user}:{passwd}@{host}:5439/{database}")
    num_iterations = (num_rows // chunk_size) + 1 if num_rows % chunk_size != 0 else num_rows // chunk_size

    for i in range(num_iterations):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size if (i + 1) * chunk_size < num_rows else num_rows

        chunk_df = dataframe.iloc[start_idx:end_idx]

        chunk_df.to_sql(schema= schema_name, name = table_name, con =conn, index = False, if_exists = 'append', chunksize = 5000, method = 'multi')

    print("Data loaded successfully.")
    

In [0]:
# Query to get all table names from information_schema.tables
#query = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'vbiz'"
#tables =   pd.read_sql(query, con=vbizdb)
#tables = [list(row) for row in tables.itertuples(index=False)]

In [0]:
tables = [
'account',
'approver',
'audit',
'bank',
'batch',
'beneficiary',
'business',
#'businessUser',
'clocks',
'document',
'leader',
'onboarding', 
#'passwordReset',
'profile',
#'refreshSession',
#'subAccountConsentApproval',
#'subAccountRequest',
'taggable',
#'user',
#'workflowRequest'
'payment',
]
print(tables)
# 'businessLeader', 'onboardingOld',  'tag',

In [0]:
print("Started: " + datetime.today().strftime('%Y-%m-%d %H:%M:%S'))

# Loop through each table and transfer data from MySQL to MSSQL
for table in tables:
    # Read data from MySQL table
    query = f"SELECT *, NOW() AS load_date FROM {table} ORDER BY CreatedAt"
    print({table})
    df = pd.read_sql(query, con=vbizdb)
    
    def truncate_string(s):
        if isinstance(s, str):
            return s[:65000]
        else:
            return s

    # Apply the truncate_string function to all columns in the DataFrame
    df = df.applymap(truncate_string)

    # Write data to MSSQL table
    load_table('vbiz', table, df)

# Close database connections
vbizdb.close()

print("Ended: " + datetime.today().strftime('%Y-%m-%d %H:%M:%S'))